In [ ]:
!pip install pdfplumber pandas

In [2]:
from google.colab import files
uploaded = files.upload()   # select your PDF file


Saving mPassbook_CA-510909010204363_1758262246131.pdf to mPassbook_CA-510909010204363_1758262246131.pdf


In [ ]:
import pdfplumber

def extract_text(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

raw_text = extract_text("mPassbook_CA-510909010204363_1758262246131.pdf")
print(raw_text[:1000])  # preview first 1000 chars

CITY UNION BANK
BRANCH : THIRUKATTUPALLI
87 Kamarajar Bazaar Street,Tirukattupalli,Tirukattupalli,Thanjavur 613104
ACCOUNT NO :CA-510909010204363
ACCOUNT NO(15 DIGIT):510909010204363
IFSC :CIUB0000019
ACCOUNT TYPE :
CUSTOMER DETAILS :SRI LUCKY HARDWARES
NO 162 THIRUKATTUPALLI MAIN ROAD
PADMA COMPLEX
LINE KARI
THIRUKATTUPALLI
613104
Statement Date :Sep 19, 2025, at 11:40 AM
STATEMENT OF ACCOUNT from 01/08/2025 to 31/08/2025
DATE DESCRIPTION CHEQUE NO DEBIT CREDIT BALANCE
01/08/2025 TO ONL UPI/DR/557958675629/SRI MURU/ICIC/EAZYPAY 7L/U::00019 10,000.00 1,09,114.06
01/08/2025 BY NEFT TRF:ONE 97 COMMUNICA YESAP52130595239: 3,520.00 1,12,634.06
01/08/2025 TO ONL TRF: OD 512020010022526:00019 15,000.00 97,634.06
01/08/2025 TO ONL UPI/DR/521332232400/RIYOM LA/HDFC/5020011016/U::00019 18,090.00 79,544.06
01/08/2025 TO ONL UPI/DR/521347340165/CITY ALU/ICIC/CITYALUMIN/U::00019 2,100.00 77,444.06
01/08/2025 TO ONL UPI/DR/521399950429/SIVA GAN/YESB/PAYTMQR649/U::00019 506.00 76,938.06
02/08/2025 B

In [ ]:
import re
import pandas as pd

def parse_transactions(text):
    data = []
    lines = text.split("\n")

    pattern = re.compile(
        r"(\d{2}/\d{2}/\d{4})\s+(.*?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)\s+(\d{1,3}(?:,\d{3})*(?:\.\d{2})?)?"
    )

    for line in lines:
        m = pattern.search(line)
        if m:
            date = m.group(1)
            desc = m.group(2).strip()
            debit = m.group(3) if "TO" in line else None
            credit = m.group(3) if "BY" in line else None
            if debit:
                amount = -float(debit.replace(",", ""))
            elif credit:
                amount = float(credit.replace(",", ""))
            else:
                amount = None

            data.append({
                "Date": date,
                "Description": desc,
                "Amount": amount,
                "Category": "Uncategorized"
            })
    return pd.DataFrame(data)

df = parse_transactions(raw_text)
df.head(10)

,Date,Description,Amount,Category
0,01/08/2025,TO ONL UPI/DR/557958675629/SRI MURU/ICIC/EAZYP...,-10000.0,Uncategorized
1,01/08/2025,BY NEFT TRF:ONE,97.0,Uncategorized
2,01/08/2025,TO ONL TRF: OD 512020010022526:00019,-15000.0,Uncategorized
3,01/08/2025,TO ONL UPI/DR/521332232400/RIYOM LA/HDFC/50200...,-18090.0,Uncategorized
4,01/08/2025,TO ONL UPI/DR/521347340165/CITY ALU/ICIC/CITYA...,-2100.0,Uncategorized
5,01/08/2025,TO ONL UPI/DR/521399950429/SIVA GAN/YESB/PAYTM...,-506.0,Uncategorized
6,02/08/2025,BY NEFT TRF:ONE,97.0,Uncategorized
7,02/08/2025,TO ONL UPI/DR/521494180726/DHANLAXM/KVBL/82485...,-30000.0,Uncategorized
8,02/08/2025,TO ONL UPI/DR/558028323317/SHEIK S/ICIC/CITYCO...,-8830.0,Uncategorized
9,03/08/2025,BY NEFT TRF:ONE,97.0,Uncategorized


In [ ]:
df.to_csv("upi_transactions.csv", index=False)
df.to_json("upi_transactions.json", orient="records", indent=2)

from google.colab import files
files.download("upi_transactions.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd

# Load your parsed bank statement
df = pd.read_csv("upi_transactions.csv")

# Ensure Date column is datetime
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Preview
df.head()

,Date,Description,Amount,Category
0,2025-01-08,TO ONL UPI/DR/557958675629/SRI MURU/ICIC/EAZYP...,-10000.0,Uncategorized
1,2025-01-08,BY NEFT TRF:ONE,97.0,Uncategorized
2,2025-01-08,TO ONL TRF: OD 512020010022526:00019,-15000.0,Uncategorized
3,2025-01-08,TO ONL UPI/DR/521332232400/RIYOM LA/HDFC/50200...,-18090.0,Uncategorized
4,2025-01-08,TO ONL UPI/DR/521347340165/CITY ALU/ICIC/CITYA...,-2100.0,Uncategorized


In [ ]:
category_keywords = {
    "Food": ["zomato", "swiggy", "hotel", "restaurant", "bakery"],
    "Travel": ["uber", "ola", "rapido", "irctc", "bus", "train", "flight", "airlines"],
    "Groceries": ["bigbasket", "grofers", "reliance fresh", "more supermarket", "dmart"],
    "Utilities": ["electricity", "gas", "water bill", "recharge", "dth", "bsnl", "airtel", "jio"],
    "Shopping": ["amazon", "flipkart", "myntra", "ajio"],
    "Income": ["salary", "credited", "neft", "rtgs", "imps", "payment received"],
    "Others": []
}

In [ ]:
def categorize(description):
    desc = str(description).lower()
    for category, keywords in category_keywords.items():
        for kw in keywords:
            if kw in desc:
                return category
    return "Others"

In [ ]:
df["Category"] = df["Description"].apply(categorize)

# Check sample
df[["Description", "Category"]].head(20)

,Description,Category
0,TO ONL UPI/DR/557958675629/SRI MURU/ICIC/EAZYP...,Others
1,BY NEFT TRF:ONE,Income
2,TO ONL TRF: OD 512020010022526:00019,Others
3,TO ONL UPI/DR/521332232400/RIYOM LA/HDFC/50200...,Others
4,TO ONL UPI/DR/521347340165/CITY ALU/ICIC/CITYA...,Others
5,TO ONL UPI/DR/521399950429/SIVA GAN/YESB/PAYTM...,Others
6,BY NEFT TRF:ONE,Income
7,TO ONL UPI/DR/521494180726/DHANLAXM/KVBL/82485...,Others
8,TO ONL UPI/DR/558028323317/SHEIK S/ICIC/CITYCO...,Others
9,BY NEFT TRF:ONE,Income


In [ ]:
import re

def clean_description(desc):
    desc = str(desc)
    # remove long UPI refs like /123456789/
    desc = re.sub(r'/\d{6,}/', ' ', desc)
    # replace multiple spaces
    desc = re.sub(r'\s+', ' ', desc).strip()
    return desc

df["Clean_Description"] = df["Description"].apply(clean_description)
df[["Description","Clean_Description"]].head(10)

,Description,Clean_Description
0,TO ONL UPI/DR/557958675629/SRI MURU/ICIC/EAZYP...,TO ONL UPI/DR SRI MURU/ICIC/EAZYPAY 7L/U::00019
1,BY NEFT TRF:ONE,BY NEFT TRF:ONE
2,TO ONL TRF: OD 512020010022526:00019,TO ONL TRF: OD 512020010022526:00019
3,TO ONL UPI/DR/521332232400/RIYOM LA/HDFC/50200...,TO ONL UPI/DR RIYOM LA/HDFC U::00019
4,TO ONL UPI/DR/521347340165/CITY ALU/ICIC/CITYA...,TO ONL UPI/DR CITY ALU/ICIC/CITYALUMIN/U::00019
5,TO ONL UPI/DR/521399950429/SIVA GAN/YESB/PAYTM...,TO ONL UPI/DR SIVA GAN/YESB/PAYTMQR649/U::00019
6,BY NEFT TRF:ONE,BY NEFT TRF:ONE
7,TO ONL UPI/DR/521494180726/DHANLAXM/KVBL/82485...,TO ONL UPI/DR DHANLAXM/KVBL U::00019
8,TO ONL UPI/DR/558028323317/SHEIK S/ICIC/CITYCO...,TO ONL UPI/DR SHEIK S/ICIC/CITYCORPOR/UP::00019
9,BY NEFT TRF:ONE,BY NEFT TRF:ONE


In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

candidate_labels = ["Food", "Travel", "Groceries", "Utilities", "Shopping", "Income", "Other"]

def categorize(desc):
    result = classifier(desc, candidate_labels)
    return result["labels"][0]

df["Category"] = df["Clean_Description"].apply(categorize)
df[["Clean_Description","Category"]].head(10)




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


,Clean_Description,Category
0,TO ONL UPI/DR SRI MURU/ICIC/EAZYPAY 7L/U::00019,Income
1,BY NEFT TRF:ONE,Travel
2,TO ONL TRF: OD 512020010022526:00019,Travel
3,TO ONL UPI/DR RIYOM LA/HDFC U::00019,Travel
4,TO ONL UPI/DR CITY ALU/ICIC/CITYALUMIN/U::00019,Utilities
5,TO ONL UPI/DR SIVA GAN/YESB/PAYTMQR649/U::00019,Income
6,BY NEFT TRF:ONE,Travel
7,TO ONL UPI/DR DHANLAXM/KVBL U::00019,Travel
8,TO ONL UPI/DR SHEIK S/ICIC/CITYCORPOR/UP::00019,Travel
9,BY NEFT TRF:ONE,Travel


In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

candidate_labels = ["Food", "Travel", "Groceries", "Utilities", "Shopping", "Income", "Other"]

def categorize(desc):
    result = classifier(desc, candidate_labels)
    return result["labels"][0]

df["Category"] = df["Clean_Description"].apply(categorize)
df[["Clean_Description","Category"]].head(10)

Device set to use cpu


,Clean_Description,Category
0,TO ONL UPI/DR SRI MURU/ICIC/EAZYPAY 7L/U::00019,Income
1,BY NEFT TRF:ONE,Travel
2,TO ONL TRF: OD 512020010022526:00019,Travel
3,TO ONL UPI/DR RIYOM LA/HDFC U::00019,Travel
4,TO ONL UPI/DR CITY ALU/ICIC/CITYALUMIN/U::00019,Utilities
5,TO ONL UPI/DR SIVA GAN/YESB/PAYTMQR649/U::00019,Income
6,BY NEFT TRF:ONE,Travel
7,TO ONL UPI/DR DHANLAXM/KVBL U::00019,Travel
8,TO ONL UPI/DR SHEIK S/ICIC/CITYCORPOR/UP::00019,Travel
9,BY NEFT TRF:ONE,Travel


In [ ]:
def detect_wasteful(row):
    if row["Category"] in ["Food","Shopping","Travel"] and abs(row["Amount"]) < 500:
        return 1   # Wasteful
    return 0       # Not Wasteful

df["Wasteful"] = df.apply(detect_wasteful, axis=1)
df[["Clean_Description","Category","Amount","Wasteful"]].head(15)


,Clean_Description,Category,Amount,Wasteful
0,TO ONL UPI/DR SRI MURU/ICIC/EAZYPAY 7L/U::00019,Income,-10000.0,0
1,BY NEFT TRF:ONE,Travel,97.0,1
2,TO ONL TRF: OD 512020010022526:00019,Travel,-15000.0,0
3,TO ONL UPI/DR RIYOM LA/HDFC U::00019,Travel,-18090.0,0
4,TO ONL UPI/DR CITY ALU/ICIC/CITYALUMIN/U::00019,Utilities,-2100.0,0
5,TO ONL UPI/DR SIVA GAN/YESB/PAYTMQR649/U::00019,Income,-506.0,0
6,BY NEFT TRF:ONE,Travel,97.0,1
7,TO ONL UPI/DR DHANLAXM/KVBL U::00019,Travel,-30000.0,0
8,TO ONL UPI/DR SHEIK S/ICIC/CITYCORPOR/UP::00019,Travel,-8830.0,0
9,BY NEFT TRF:ONE,Travel,97.0,1


In [ ]:
print("Wasteful %:", (df["Wasteful"]=="Wasteful").mean()*100)

print("\nWasteful transactions:")
print(df[df["Wasteful"]=="Wasteful"].head(10))

Wasteful %: 0.0

Wasteful transactions:
Empty DataFrame
Columns: [Date, Description, Amount, Category, Clean_Description, Wasteful]
Index: []


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F
0
model_name = "facebook/bart-large-mnli"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [ ]:
def entailment_score(premise, hypothesis):
    # Encode the text pair
    inputs = tokenizer(premise, hypothesis, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    # Order: ['contradiction', 'neutral', 'entailment']
    entail_contradiction_logits = logits[:, [0, 2]]
    probs = F.softmax(entail_contradiction_logits, dim=1)
    return probs[:, 1].item()  # probability of entailment


In [ ]:
candidate_labels = ["Food", "Travel", "Groceries", "Utilities", "Shopping", "Income", "Other"]

def categorize_pair(desc):
    scores = {}
    for label in candidate_labels:
        hypothesis = f"This transaction is about {label}."
        score = entailment_score(desc, hypothesis)
        scores[label] = score
    best_label = max(scores, key=scores.get)   # pick label with highest score
    return best_label, scores

df["Category_Pair"], df["Category_Scores"] = zip(*df["Clean_Description"].apply(categorize_pair))


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr

candidate_labels = ["Food", "Travel", "Groceries", "Utilities", "Shopping", "Income", "Other"]

# Categorization with text-pair modeling
def categorize_pair(desc):
    scores = {}
    for label in candidate_labels:
        hypothesis = f"This transaction is about {label}."
        score = entailment_score(desc, hypothesis)
        scores[label] = score
    best_label = max(scores, key=scores.get)
    return best_label

# Wasteful / Not Wasteful classification
def wasteful_classification(desc):
    hypotheses = {
        "Wasteful": "This transaction is a wasteful expense.",
        "Not Wasteful": "This transaction is not a wasteful expense."
    }
    scores = {label: entailment_score(desc, hypo) for label, hypo in hypotheses.items()}
    return max(scores, key=scores.get)

# Gradio function
def analyze_transaction(desc, amount):
    category = categorize_pair(desc)
    wasteful = wasteful_classification(desc)
    return category, wasteful

# Build Gradio Interface
demo = gr.Interface(
    fn=analyze_transaction,
    inputs=[gr.Textbox(label="Transaction Description"),
            gr.Number(label="Amount")],
    outputs=[gr.Label(label="Predicted Category"),
             gr.Label(label="Wasteful / Not Wasteful")],
    title="💳 UPI Analyzer (Text Pair Modeling)",
    description="Classify bank transactions into categories and detect wasteful spends using text-pair modeling."
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d7db7b880c10771fd2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
